In [1]:
# install MeCab
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
# check path to "ipadic-neologd" 
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [3]:
# Test
import MeCab
m = MeCab.Tagger()
sample_txt = "北斗の拳の原作者は武論尊。"
print("Mecab:\n", m.parse(sample_txt))

path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(path)
print("Mecab ipadic NEologd:\n",m.parse(sample_txt))

Mecab:
 北斗	名詞,一般,*,*,*,*,北斗,ホクト,ホクト
の	助詞,連体化,*,*,*,*,の,ノ,ノ
拳	名詞,一般,*,*,*,*,拳,コブシ,コブシ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
原作	名詞,一般,*,*,*,*,原作,ゲンサク,ゲンサク
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
武	名詞,一般,*,*,*,*,武,ブ,ブ
論	名詞,接尾,一般,*,*,*,論,ロン,ロン
尊	名詞,一般,*,*,*,*,尊,ミコト,ミコト
。	記号,句点,*,*,*,*,。,。,。
EOS

Mecab ipadic NEologd:
 北斗の拳	名詞,固有名詞,一般,*,*,*,北斗の拳,ホクトノケン,ホクトノケン
の	助詞,連体化,*,*,*,*,の,ノ,ノ
原作者	名詞,固有名詞,一般,*,*,*,原作者,ゲンサクシャ,ゲンサクシャ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
武論尊	名詞,固有名詞,人名,一般,*,*,武論尊,ブロンソン,ブロンソン
。	記号,句点,*,*,*,*,。,。,。
EOS



In [4]:
# matplotlibの日本語化
!pip install japanize-matplotlib

     |████████████████████████████████| 4.1MB 2.7MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.0.5-cp36-none-any.whl size=4118721 sha256=947111ac24265504c5cdf7d20bdbc8e327da91c94065d93da0a5072a3c004b22
  Stored in directory: /root/.cache/pip/wheels/6c/8a/08/4a784957da9f3c2b4839b4986be2fba2a481877318948be52c
Successfully built japanize-matplotlib


In [5]:
# Google Driveへマウント
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
%cd /content/drive/'My Drive'/'Google Colab'/'industryPrediction'/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Google Colab/industryPrediction


In [6]:
!pip install japanize-matplotlib

In [0]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

# スコア評価用ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support

# エラー表示の抑制
import warnings

In [0]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [9]:
# テキストデータの読み込み
wikiData = pd.read_csv("csv/train_cleaned.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧 南ヶ丘牧（みなみがおかぼくじょう）は、栃那須原に拠を置く酪農経営企。現在4つ...,1
2,岩瀬牧 岩瀬牧（いわせぼくじょう）は、福島岩瀬郡鏡石にある牧である。 鏡石シンボルソン...,1
3,"クラギ クラギ（くらぎ、""KURAGI Co.Ltd."" ）は、重を心に「農」を展開する...",1
4,"ハッピーネモファーム ハッピーネモファーム（""""）は、北海浦河郡浦河にある競走馬（サラブ...",1


In [0]:
# Numpy配列へ変換
X = wikiData.text.values
y = wikiData.target.values

In [0]:
# トレーニングデータ、テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42, stratify=y)

In [0]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger()
    tagger.parse("")
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in ['形容詞', '動詞','名詞']:
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [0]:
# 形態素解析を実行
vectorizer = TfidfVectorizer(tokenizer=tokenize)
vectorizer.fit(X)
train_matrix = vectorizer.transform(X_train)
test_matrix = vectorizer.transform(X_test)

In [16]:
# ナイーブベイズ
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf, train_matrix, y_train, cv=5)

print("Training score：" + str(clf.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf.score(test_matrix, y_test)))

Training score：0.4640052356020942
Cross-Validation score：0.36965752412303965
Test score：0.3717277486910995


In [17]:
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier(n_estimators=50,random_state=42)
clf2.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf2, train_matrix, y_train, cv=5)

print("Training score：" + str(clf2.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf2.score(test_matrix, y_test)))

Training score：1.0
Cross-Validation score：0.6497652657515415
Test score：0.6426701570680629


In [18]:
# Light GBM
import lightgbm as lgb

clf3 = lgb.LGBMClassifier(objective='multiclass', num_class=33)
clf3.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf3, train_matrix, y_train, cv=5)

print("Training score：" + str(clf3.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf3.score(test_matrix, y_test)))

Training score：0.05770069808027923
Cross-Validation score：0.10504214668642597
Test score：0.051047120418848166


In [19]:
# SVM
from sklearn.svm import SVC

## パラメーター設定
C = 1.
kernel = 'rbf'
gamma  = 0.01

## One-versus-the-rest
clf4 = SVC(C=C, kernel=kernel, gamma=gamma, decision_function_shape='ovr')
clf4.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf4, train_matrix, y_train, cv=5)

print("Training score：" + str(clf4.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf4.score(test_matrix, y_test)))


## One-versus-the-one(multi-class default)
clf5 = SVC(C=C, kernel=kernel, gamma=gamma, decision_function_shape='ovo')
clf5.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf5, train_matrix, y_train, cv=5)

print("Training score：" + str(clf5.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf5.score(test_matrix, y_test)))

Training score：0.20353403141361257
Cross-Validation score：0.18465512121501293
Test score：0.19589877835951133
Training score：0.20353403141361257
Cross-Validation score：0.18465512121501293
Test score：0.19589877835951133
